In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from lid_driven_cavity import states, operators, postprocessing

In [3]:
np.set_printoptions(linewidth=2000)

In [4]:
N = 3
Nx = N+1
Ny = N+2
n = Nx*Ny
h = 1 / Nx
operator = operators.Operator((n, n))
hm2 = h**(-2)
b = np.zeros((Ny*Nx,))

# loop on internal grid points for now
for i in range(1, Nx-1):
    for j in range(1, Ny-1):

        # pointers
        row = states.lij(i, j, Nx)    # this is the row
        ip1 = states.lij(i+1, j, Nx)  # (i+1,j)
        im1 = states.lij(i-1, j, Nx)  # (i-1,j)
        jp1 = states.lij(i, j+1, Nx)  # (i,j+1)
        jm1 = states.lij(i, j-1, Nx)  # (i,j-1)

        # add coefficients
        operator.append(row, row, -4*hm2)  # diagonal
        operator.append(row, ip1, hm2)
        operator.append(row, im1, hm2)
        operator.append(row, jp1, hm2)
        operator.append(row, jm1, hm2)

# top and bottom surfaces
jtop = Ny-1
jbottom = 0
for i in range(0, Nx):

    # top surface (y=1)
    row = states.lij(i, jtop, Nx)  # this is the row
    jm1 = states.lij(i, jtop-1, Nx)  # (i,j-1)
    operator.append(row, row, 1)
    operator.append(row, jm1, 1)
    b[row] = 2  # TODO add gradient terms

    # bottom surface (y=0)
    row = states.lij(i, jbottom, Nx)  # this is the row
    jp1 = states.lij(i, jbottom+1, Nx)  # (i,j-1)
    operator.append(row, row, 1)
    operator.append(row, jp1, 1)
    b[row] = 0  # TODO add gradient terms

# left and right surfaces
iright = Nx-1
ileft = 0
for j in range(1, Ny-1):

    # right surface (x=1)
    row = states.lij(iright, j, Nx)  # this is the row
    operator.append(row, row, 1)
    b[row] = 0  # TODO add gradient terms

    # left surface (x=0)
    row = states.lij(ileft, j, Nx)  # this is the row
    operator.append(row, row, 1)
    b[row] = 0  # TODO add gradient terms


In [5]:
print(operator.get_csr().toarray())

[[  1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  16.   0.   0.  16. -64.  16.   0.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  16.   0.   0.  16. -64.  16.   0.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  16.   0.   0.  16. -64.  16.   0.   0.  16.  